In [44]:
import sys
import os

# Add the path to the aima-python repo to sys.path
sys.path.append(os.path.abspath('/Users/lukegosnell/Downloads/CUYear5/AI/FinalProject/aima_python'))

from aima_python.games4e import Game
from aima_python.utils4e import vector_add, MCT_Node, ucb

import random
random.seed(109)

In [122]:
class Mancala(Game):
    def __init__(self, pits_per_player=6, stones_per_pit=4):
        """
        The constructor for the Mancala class defines several instance variables:

        pits_per_player: This variable stores the number of pits each player has.
        stones_per_pit: It represents the number of stones each pit contains at the start of any game.
        board: This data structure is responsible for managing the Mancala board.
        current_player: This variable takes the value 1 or 2, as it's a two-player game, indicating which player's turn it is.
        moves: This is a list used to store the moves made by each player. It's structured in the format (current_player, chosen_pit).
        p1_pits_index: A list containing two elements representing the start and end indices of player 1's pits in the board data structure.
        p2_pits_index: Similar to p1_pits_index, it contains the start and end indices for player 2's pits on the board.
        p1_mancala_index and p2_mancala_index: These variables hold the indices of the Mancala pits on the board for players 1 and 2, respectively.
        """
        self.pits_per_player = pits_per_player
        self.board = [stones_per_pit] * ((pits_per_player+1) * 2)  # Initialize each pit with stones_per_pit number of stones 
        self.players = 2
        self.current_player = 1
        self.moves = []
        self.p1_pits_index = [0, self.pits_per_player - 1]
        self.p1_mancala_index = self.pits_per_player
        self.p2_pits_index = [self.pits_per_player+1, 2 * self.pits_per_player - 1]
        self.p2_mancala_index = 2 * self.pits_per_player
        
        # Zeroing the Mancala for both players
        self.board[self.p1_mancala_index] = 0
        self.board[self.p2_mancala_index] = 0

        self.initial = self.board  # Setting the initial state

    def actions(self, state):
        """Return a collection of the allowable moves from this state."""
        valid_pits = []
        if self.current_player == 1:
            pits = range(self.p1_pits_index[0], self.p1_pits_index[1] + 1)
        else:
            pits = range(self.p2_pits_index[0], self.p2_pits_index[1] + 1)
        
        for pit in pits:
            if self.board[pit] > 0:
                valid_pits.append(pit)  # Return pits as 1-based index
        return valid_pits

    def result(self, state, move):
        """Return the state that results from making a move from a state."""
        new_game = Mancala(self.pits_per_player)
        new_game.board = state[:]
        new_game.play(move)
        return new_game.board

    def is_terminal(self, state):
        """Return True if this is a final state for the game."""
        return self.winning_eval()

    def utility(self, state, player):
        """Return the value of this final state to player."""
        if player == 1:
            return self.board[self.p1_mancala_index] - self.board[self.p2_mancala_index]
        elif player == 2:
            return self.board[self.p2_mancala_index] - self.board[self.p1_mancala_index]
        return 0  # Default return value

    def display_board(self):
        """
        Displays the board
        """
        player_1_mancala = self.board[self.p1_mancala_index]
        player_2_mancala = self.board[self.p2_mancala_index]
        player_1_pits = self.board[:self.pits_per_player]
        player_2_pits = self.board[self.pits_per_player+1: self.pits_per_player + self.pits_per_player + 1]

        print('P2   1   2   3   4   5   6   P1')
        print('    | {} | {} | {} | {} | {} | {} |'.format(*player_1_pits))
        print('{}                              {}'.format(player_2_mancala, player_1_mancala))
        print('    | {} | {} | {} | {} | {} | {} |'.format(*player_2_pits))
        print('P2   6   5   4   3   2   1   P1')

    def valid_move(self, pit):
        """
        Function to check if the pit chosen by the current_player is a valid move.
        """
        current_player = self.current_player
        
        if pit < 1:
            print("Invalid pit selected")
            return False
        
        if current_player == 1:
            pit_index = pit - 1
        elif current_player == 2:
            pit_index = self.pits_per_player + pit
            
        if pit_index == self.p1_mancala_index or pit_index == self.p2_mancala_index:
            print("Cannot play from a mancala")
            return False
        
        if self.board[pit_index] <= 0:
            print("Cannot play from a pit with no stones")
            return False
        
        self.moves.append((current_player, pit))
        return True

    def play(self, pit):
        """
        This function simulates a single move made by a specific player using their selected pit.
        """
        
        if not self.valid_move(pit):
            print("INVALID MOVE")
            return
        
        if self.winning_eval() == True:
            print("GAME OVER")
            return
        
        # Check for the current player
        current_player = self.current_player
        
        # Assign the pit_index based on the player
        if current_player == 1:
            pit_index = pit - 1
        elif current_player == 2:
            pit_index = self.pits_per_player + pit
        
        # Pick up the stones
        stones = self.board[pit_index]
        # Remove them from the selected pit
        self.board[pit_index] = 0
        
        index = pit_index
        
        # While there are still available stones
        while stones > 0:
            # Wrap around the board
            index = (index + 1) % len(self.board)
            
            # Skip the opponents mancala
            if (current_player == 1 and index == self.p2_mancala_index) or (current_player == 2 and index == self.p1_mancala_index):
                continue
                
            # Put a stone in the pit
            self.board[index] += 1
            # Take a stone from the player
            stones -= 1
            
        if current_player == 1 and self.p1_pits_index[0] <= index <= self.p1_pits_index[1] and self.board[index] == 1:
            opposite_index = 12 - index
            
            if self.board[opposite_index] > 0:
                # Print out the number of stones here
                print(f"Player 1 captures stones from Player 2 pit {opposite_index - 6}.")
                
                if self.board[index] > self.pits_per_player + 2: #Check if it went all the way around the board
                    self.board[self.p1_mancala_index] += self.board[opposite_index] + 1
                    self.board[index] = 0
                    self.board[opposite_index] = 0
                
                
        elif current_player == 2 and self.p2_pits_index[0] <= index <= self.p2_pits_index[1] and self.board[index] == 1:
            opposite_index = 12 - index
            
            if self.board[opposite_index] > 0:
                # Print out the number of stones here
                print(f"Player 2 captures stones from Player 1 pit {opposite_index}.")
                
                if self.board[index] > self.pits_per_player + 2: #Check if it went all the way around the board
                    self.board[self.p2_mancala_index] += self.board[opposite_index] + 1
                    self.board[index] = 0
                    self.board[opposite_index] = 0
                
        
        if (current_player == 1):
            self.current_player = 2
        else:
            self.current_player = 1
            
        print(f"Turn: Player {self.current_player}")
    
    def winning_eval(self):
        """
        Function to verify if the game board has reached the winning state.
        """
        p1_pits_empty = all(stones == 0 for stones in self.board[self.p1_pits_index[0]:self.p1_pits_index[1] + 1])
        p2_pits_empty = all(stones == 0 for stones in self.board[self.p2_pits_index[0]:self.p2_pits_index[1] + 1])
        return p1_pits_empty or p2_pits_empty

In [124]:
# Create a new game instance
game = Mancala()

# Test move 1: Player 1 makes a move from pit 1
print("\nPlayer 1 makes a move from pit 1:")
game.play(1)
print("\nPlayer 2 makes a move from pit 1:")
game.play(1)
print("\nPlayer 1 makes a move from pit 2:")
game.play(2)
print("\nPlayer 2 makes a move from pit 2:")
game.play(2)
print("\nPlayer 1 makes a move from pit 3:")
game.play(3)
print("\nPlayer 2 makes a move from pit 3:")
game.play(3)
print("\nPlayer 1 makes a move from pit 4:")
game.play(4)
print("\nPlayer 2 makes a move from pit 4:")
game.play(4)
print("\nPlayer 1 makes a move from pit 5:")
game.play(5)
print("\nPlayer 2 makes a move from pit 5:")
game.play(5)
game.display_board()


Player 1 makes a move from pit 1:
Turn: Player 2

Player 2 makes a move from pit 1:
Turn: Player 1

Player 1 makes a move from pit 2:
Turn: Player 2

Player 2 makes a move from pit 2:
Turn: Player 1

Player 1 makes a move from pit 3:
Turn: Player 2

Player 2 makes a move from pit 3:
Turn: Player 1

Player 1 makes a move from pit 4:
Turn: Player 2

Player 2 makes a move from pit 4:
Turn: Player 1

Player 1 makes a move from pit 5:
Turn: Player 2

Player 2 makes a move from pit 5:
Turn: Player 1
P2   1   2   3   4   5   6   P1
    | 4 | 3 | 2 | 2 | 1 | 9 |
4                              4
    | 4 | 3 | 2 | 1 | 0 | 4 |
P2   6   5   4   3   2   1   P1
